In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

In [5]:
def read_data_from_subjects(index):
    dataset = []
    folder = '../data/subjects-xy/'
    files = os.listdir(folder)

    for f in files:
        data = pd.read_csv(folder + f, delimiter=";").to_numpy()
        positions = data[:,index]
        dataset.append(positions)

    return dataset

In [13]:
#Full data acquired here:
dataset = pd.read_csv('../data/full_data.csv')

#Dataset:
#Timestamp
#Lap_Number (0,1)
#Position.x., Position.y., Position.z.
#Orientation.y. (0-360, yaw rotation)
#Gaze_Location.x., .y., .z. (original 3D gaze point)
#Subject
#Gaze.x. (0-2560, screen-relative gaze coord)
#Gaze.y. (0-1080, screen-relative gaze coord)
#d_ang (change in Orientation.y.)
#ang_accel (change in d_ang)
#Gate_N (Upcoming gate #, can substitute for gaze point)
#Ang_Pos (-360:360, angle between Orientation.y. and normal vector of upcoming gate)

In [11]:
#Normalize (x,y) to 0-1
gaze_x = dataset['Gaze.x.'] / 2560
gaze_y = dataset['Gaze.y.'] / 1080

gaze_x.max(), gaze_y.max()

In [35]:
positions = read_data_from_subjects([2, 4])
gaze_x = read_data_from_subjects(-2)
gaze_y = read_data_from_subjects(-1)

gaze_x[1]
gaze_y[1]

max(gaze_x[0])

358954.2

In [ ]:
def read_data_from_set():
    